In [0]:
!pip install cartoframes==1.0b3
!pip install geopandas



In [0]:
import geopandas as gpd
from cartoframes.viz import Map, Layer, Legend, Source
from shapely import wkt
import geojson

In [0]:
!pip install h3

In [0]:
import folium
from h3 import h3
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [18]:


h3_address = h3.geo_to_h3(47.3615593, -122.0553238, 9) # lat, lng, hex resolution                                                                                                        
m = visualize_hexagons([h3_address])
display(m)



In [0]:
!pip install geojson

In [0]:
!pip install area

In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
from area import area

import copy

In [20]:
import requests 
url = 'https://raw.githubusercontent.com/cnilsen/EEwPython/master/KingCo/KingCoBounds.geojson'

r = requests.get(url, allow_redirects=True)
open('bounds.geojson', 'wb').write(r.content)

667407

In [8]:
with open("bounds.geojson") as f:
    bounds = json.load(f)
    
kingCo = pd.DataFrame(json_normalize(bounds))
kingCo.head(5)




,type,coordinates
0,Polygon,"[[[-122.14363243499282, 47.25781018157686], [-..."


In [21]:
#take the first and only row of the dataframe
row_sel = kingCo.iloc[0]
print(row_sel)


type                                                     Polygon
coordinates    [[[-122.14363243499282, 47.25781018157686], [-...
Name: 0, dtype: object


In [22]:
set_hexagons = h3.polyfill(row_sel, res = 9, geo_json_conformant = True)
list_hexagons = list(set_hexagons)
print("the subzone was filled with ", len(list_hexagons), "hexagons at resolution 9")

the subzone was filled with  63607 hexagons at resolution 9


In [38]:
map_subzone = folium.Map(location= [47.3615593, -122.0553238], zoom_start=11, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
folium.GeoJson(
        bounds,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 5,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_subzone)


#map_subzone.save('source/6_subzone.html')
map_subzone

### 12. Compacting the hexagons that fill a polygon

In [53]:
list_hexagons_compacted = list(h3.compact(list_hexagons))

df_fill_hex_compacted = pd.DataFrame({"hex_id": list_hexagons_compacted})
df_fill_hex_compacted["value"] = 0

#see what resolutions have the compacted hexagons
df_fill_hex_compacted["resolution"] = df_fill_hex_compacted["hex_id"].apply(lambda x: h3.h3_get_resolution(x))

df_fill_hex_compacted["resolution"].value_counts()




9    1300
8     487
7     166
6      43
5       8
4       1
Name: resolution, dtype: int64

So there are 14 hexagons at resolution 7, 45 hexagons at resolution 8 and 193 hexagons at resolution 9 to fill this ity subzone's polygon

###functions

In [0]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [0]:
def reverse_lat_lon(hex_coords):
    geom_hex = []
    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1],lat_lon[0]])
        
    return geom_hex

## test


In [70]:
df_fill_hex_compacted['geometry'] = df_fill_hex_compacted.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=x,geo_json=False))]
                                                        }
                                                    )

geojson_hx_compacted = hexagons_dataframe_to_geojson(df_fill_hex_compacted)

geojson_hx_compacted

'{"type": "FeatureCollection", "features": [{"type": "Feature", "id": "8928d0d262fffff", "geometry": {"type": "Polygon", "coordinates": [[[-121.107701, 47.692601], [-121.106478, 47.694133], [-121.107867, 47.695463], [-121.11048, 47.695262], [-121.111704, 47.69373], [-121.110314, 47.6924]]]}, "properties": {"value": 0}}, {"type": "Feature", "id": "8928d55598bffff", "geometry": {"type": "Polygon", "coordinates": [[[-122.44035, 47.617963], [-122.43916, 47.619505], [-122.440559, 47.620816], [-122.443148, 47.620586], [-122.444337, 47.619044], [-122.442938, 47.617733]]]}, "properties": {"value": 0}}, {"type": "Feature", "id": "8928d477597ffff", "geometry": {"type": "Polygon", "coordinates": [[[-121.446122, 47.376543], [-121.444911, 47.378083], [-121.446298, 47.379415], [-121.448896, 47.379208], [-121.450106, 47.377668], [-121.448719, 47.376335]]]}, "properties": {"value": 0}}, {"type": "Feature", "id": "8828d08169fffff", "geometry": {"type": "Polygon", "coordinates": [[[-121.576523, 47.76753

In [59]:
map_subzone_c = folium.Map(location= [47.3615593, -122.0553238], zoom_start=11, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )

map_subzone_c

In [42]:



folium.GeoJson(
        bounds,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 5,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_subzone_c)






folium.GeoJson(
        geojson_hx_compacted,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons compacted" 
    ).add_to(map_subzone_c)

folium.map.LayerControl('bottomright', collapsed=False).add_to(map_subzone_c)


#ap_subzone_c.save('source/10_compacted.html')
map_subzone_c

## uncompact those

In [0]:
list_hexagons_compacted

In [0]:
list_hexagons_uncompacted = h3.uncompact(list_hexagons_compacted,10)
list_hexagons_uncompacted

In [0]:
df_fill_hex_uncompacted = pd.DataFrame({"hex_id": list_hexagons_uncompacted})

df_fill_hex_uncompacted["value"] = 0

In [89]:
df_fill_hex_uncompacted


,hex_id,value
0,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
1,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
2,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
3,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
4,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
...,...,...
445244,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
445245,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
445246,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0
445247,"{8a28d43a5207fff, 8a28d55ac877fff, 8a28d4a9a8d...",0


In [84]:
df_fill_hex_uncompacted['geometry'] = df_fill_hex_uncompacted.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=x,geo_json=False))]
                                                        }
                                                    )



TypeError: ignored

In [68]:
dfUncomptacted = h3.uncompact(list_hexagons_compacted,9)


df_fill_hex_uncompacted['geometry'] = df_fill_hex_uncompacted.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=x,geo_json=False))]
                                                        }
                                                    )

geojson_hx_compacted = hexagons_dataframe_to_geojson(df_fill_hex_compacted)

geojson_hx_uncompacted = hexagons_dataframe_to_geojson(dfUncomptacted)

AttributeError: ignored

In [67]:
folium.GeoJson(
        dfUncomptacted,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'blue',
            'weight': 5,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_subzone_c)

ValueError: ignored